In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Glucocorticoid_Sensitivity"
cohort = "GSE42002"

# Input paths
in_trait_dir = "../../input/GEO/Glucocorticoid_Sensitivity"
in_cohort_dir = "../../input/GEO/Glucocorticoid_Sensitivity/GSE42002"

# Output paths
out_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/GSE42002.csv"
out_gene_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/gene_data/GSE42002.csv"
out_clinical_data_file = "../../output/preprocess/Glucocorticoid_Sensitivity/clinical_data/GSE42002.csv"
json_path = "../../output/preprocess/Glucocorticoid_Sensitivity/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Parse and analyze the dataset

# 1. Determine if gene expression data is available
# Based on the background information, this dataset contains gene expression arrays
# measuring mRNA expression, which indicates gene expression data is available
is_gene_available = True

# 2. Identify data availability for trait, age, and gender

# 2.1 Trait data (Glucocorticoid_Sensitivity) can be derived from the condition (trauma/control)
# Looking at sample characteristics, we can see condition info at key 1
trait_row = 1

# Convert trait data (trauma/control) to binary values (0/1)
def convert_trait(value):
    if value is None:
        return None
    
    # Extract value after colon if present
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Convert to binary (trauma=1, control=0)
    if 'trauma' in value.lower():
        return 1
    elif 'control' in value.lower():
        return 0
    else:
        return None

# 2.2 Age data is not available in the sample characteristics
age_row = None

def convert_age(value):
    # Function defined but not used since age data is unavailable
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    try:
        return float(value)
    except:
        return None

# 2.3 Gender data is not available in the sample characteristics
gender_row = None

def convert_gender(value):
    # Function defined but not used since gender data is unavailable
    if value is None:
        return None
    
    if ':' in value:
        value = value.split(':', 1)[1].strip()
        
    value = value.lower()
    if 'female' in value or 'f' == value:
        return 0
    elif 'male' in value or 'm' == value:
        return 1
    else:
        return None

# 3. Determine trait data availability
is_trait_available = trait_row is not None

# Save initial metadata about dataset usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. If trait data is available, extract and save clinical features
if trait_row is not None:
    # Parse the sample characteristics dictionary from the text representation
    sample_chars_dict = {0: ['genotype: rs1360780 AA/AG', 'genotype: rs1360780 GG'], 
                       1: ['condition: trauma', 'condition: control'], 
                       2: ['tissue: whole blood']}
    
    # Create the clinical dataframe correctly for geo_select_clinical_features
    clinical_data = pd.DataFrame()
    for key, values in sample_chars_dict.items():
        # Create a series for each row
        clinical_data[key] = values
    
    # Create a clinical dataframe using the library function
    clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the clinical dataframe
    preview = preview_df(clinical_df)
    print("Clinical Data Preview:")
    print(preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Dataset Analysis and Clinical Feature Extraction

In [ ]:
# Import necessary libraries
import pandas as pd
import os
import json
import gzip
from typing import Optional, Callable, Dict, Any

# Step 3: Analyze the dataset and extract clinical features

# Function to extract sample characteristics from GEO series matrix file
def extract_sample_characteristics(file_path):
    with gzip.open(file_path, 'rt') as f:
        lines = []
        in_characteristics = False
        geo_accessions = []
        sample_titles = []
        
        for line in f:
            if line.startswith('!Sample_characteristics_ch1'):
                in_characteristics = True
                lines.append(line.strip())
            elif in_characteristics and line.startswith('!Sample_'):
                if not line.startswith('!Sample_characteristics_ch'):
                    in_characteristics = False
                else:
                    lines.append(line.strip())
            elif line.startswith('!Sample_geo_accession'):
                geo_accessions = line.strip().split('\t')[1:]
            elif line.startswith('!Sample_title'):
                sample_titles = line.strip().split('\t')[1:]
        
        # Create a dictionary to store characteristics
        characteristic_dict = {}
        for i, line in enumerate(lines):
            parts = line.strip().split('\t')
            characteristic_dict[i] = parts[1:]
        
        # Create DataFrame
        characteristics_df = pd.DataFrame(characteristic_dict)
        if geo_accessions:
            characteristics_df.index = geo_accessions
        return characteristics_df, sample_titles

# Function to detect if the file contains gene expression data
def has_gene_expression(file_path):
    with gzip.open(file_path, 'rt') as f:
        for line in f:
            if line.startswith('!Series_platform_id'):
                platform = line.strip().split("\t")[1]
                # Check if platform is a gene expression platform (typically GPL*)
                if platform.startswith('GPL'):
                    # Gene expression platforms, not miRNA or methylation specific
                    return True
            if line.startswith('!Series_summary') or line.startswith('!Series_title'):
                # Check summary for indications this is gene expression data
                summary = line.strip()
                if 'miRNA' in summary or 'methylation' in summary:
                    return False
            if line.startswith('!platform_technology'):
                tech = line.strip().split("\t")[1].lower()
                if 'expression' in tech and not ('mirna' in tech or 'methylation' in tech):
                    return True
                if 'mirna' in tech or 'methylation' in tech:
                    return False
            # Stop after a reasonable number of lines if we haven't found definitive info
            if line.startswith('!series_matrix_table_begin'):
                break
    # Default to True if we couldn't determine otherwise
    return True

# Find and process the series matrix file
matrix_file = os.path.join(in_cohort_dir, 'GSE42002_series_matrix.txt.gz')
is_gene_available = has_gene_expression(matrix_file)

# Extract sample characteristics
clinical_data, sample_titles = extract_sample_characteristics(matrix_file)
print("Sample Characteristics:")
for i in range(len(clinical_data.columns)):
    print(f"Row {i}: {clinical_data[i].unique()}")

# Based on the data exploration, determine trait, age, and gender availability
# The data shows the following rows:
# Row 0: genotype rs1360780 (AA/AG vs GG)
# Row 1: condition (trauma vs control)
# Row 2: tissue (whole blood)

# There is no direct glucocorticoid sensitivity measure in this dataset
trait_row = None  # No direct measure of glucocorticoid sensitivity
age_row = None    # No age information
gender_row = None # No gender information

# Define conversion functions (though they won't be used in this case)
def convert_trait(value: str) -> Optional[float]:
    """Convert glucocorticoid sensitivity value to float."""
    if pd.isna(value) or value is None:
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # For GSE42002, we have no direct measure of glucocorticoid sensitivity
    return None

def convert_age(value: str) -> Optional[float]:
    """Convert age value to float."""
    if pd.isna(value) or value is None:
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Try to convert to float
    try:
        # Remove any 'years' or other text
        value = value.lower().replace('years', '').replace('year', '').strip()
        value = value.split()[0]  # Take first token if there are multiple
        return float(value)
    except:
        return None

def convert_gender(value: str) -> Optional[int]:
    """Convert gender value to binary (0 for female, 1 for male)."""
    if pd.isna(value) or value is None:
        return None
    # Extract value after colon if present
    if ':' in str(value):
        value = value.split(':', 1)[1].strip()
    
    # Convert to lowercase for case-insensitive comparison
    value = value.lower()
    
    if 'female' in value or 'f' == value or 'f ' in value:
        return 0
    elif 'male' in value or 'm' == value or 'm ' in value:
        return 1
    return None

# Check if trait data is available
is_trait_available = trait_row is not None

# Save initial metadata
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=is_trait_available
)

# Process clinical features if trait data is available
# In this case, it's not available so we skip this step
if is_trait_available:
    def get_feature_data(df, row_idx, feature_name, convert_func):
        """Helper function to extract feature data and convert to appropriate type."""
        feature_data = df[row_idx].apply(convert_func).rename(feature_name)
        return pd.DataFrame(feature_data)
        
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age if age_row is not None else None,
        gender_row=gender_row,
        convert_gender=convert_gender if gender_row is not None else None
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Save the clinical data
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")
else:
    print("No trait data available for this cohort. Skipping clinical feature extraction.")


### Step 4: Gene Data Extraction

In [ ]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. First, let's examine the structure of the matrix file to understand its format
import gzip

# Peek at the first few lines of the file to understand its structure
with gzip.open(matrix_file, 'rt') as file:
    # Read first 100 lines to find the header structure
    for i, line in enumerate(file):
        if '!series_matrix_table_begin' in line:
            print(f"Found data marker at line {i}")
            # Read the next line which should be the header
            header_line = next(file)
            print(f"Header line: {header_line.strip()}")
            # And the first data line
            first_data_line = next(file)
            print(f"First data line: {first_data_line.strip()}")
            break
        if i > 100:  # Limit search to first 100 lines
            print("Matrix table marker not found in first 100 lines")
            break

# 3. Now try to get the genetic data with better error handling
try:
    gene_data = get_genetic_data(matrix_file)
    print(gene_data.index[:20])
except KeyError as e:
    print(f"KeyError: {e}")
    
    # Alternative approach: manually extract the data
    print("\nTrying alternative approach to read the gene data:")
    with gzip.open(matrix_file, 'rt') as file:
        # Find the start of the data
        for line in file:
            if '!series_matrix_table_begin' in line:
                break
                
        # Read the headers and data
        import pandas as pd
        df = pd.read_csv(file, sep='\t', index_col=0)
        print(f"Column names: {df.columns[:5]}")
        print(f"First 20 row IDs: {df.index[:20]}")
        gene_data = df


### Step 5: Gene Identifier Review

In [ ]:
# Examining gene identifiers in the expression data
# The identifiers start with "ILMN_" which indicates these are Illumina probe IDs
# These are not human gene symbols but Illumina BeadChip probe identifiers
# They need to be mapped to standard gene symbols for better interpretability and analysis

requires_gene_mapping = True


### Step 6: Gene Annotation

In [ ]:
# 1. Let's first examine the structure of the SOFT file before trying to parse it
import gzip

# Look at the first few lines of the SOFT file to understand its structure
print("Examining SOFT file structure:")
try:
    with gzip.open(soft_file, 'rt') as file:
        # Read first 20 lines to understand the file structure
        for i, line in enumerate(file):
            if i < 20:
                print(f"Line {i}: {line.strip()}")
            else:
                break
except Exception as e:
    print(f"Error reading SOFT file: {e}")

# 2. Now let's try a more robust approach to extract the gene annotation
# Instead of using the library function which failed, we'll implement a custom approach
try:
    # First, look for the platform section which contains gene annotation
    platform_data = []
    with gzip.open(soft_file, 'rt') as file:
        in_platform_section = False
        for line in file:
            if line.startswith('^PLATFORM'):
                in_platform_section = True
                continue
            if in_platform_section and line.startswith('!platform_table_begin'):
                # Next line should be the header
                header = next(file).strip()
                platform_data.append(header)
                # Read until the end of the platform table
                for table_line in file:
                    if table_line.startswith('!platform_table_end'):
                        break
                    platform_data.append(table_line.strip())
                break
    
    # If we found platform data, convert it to a DataFrame
    if platform_data:
        import pandas as pd
        import io
        platform_text = '\n'.join(platform_data)
        gene_annotation = pd.read_csv(io.StringIO(platform_text), delimiter='\t', 
                                      low_memory=False, on_bad_lines='skip')
        print("\nGene annotation preview:")
        print(preview_df(gene_annotation))
    else:
        print("Could not find platform table in SOFT file")
        
        # Try an alternative approach - extract mapping from other sections
        with gzip.open(soft_file, 'rt') as file:
            for line in file:
                if 'ANNOTATION information' in line or 'annotation information' in line:
                    print(f"Found annotation information: {line.strip()}")
                if line.startswith('!Platform_title') or line.startswith('!platform_title'):
                    print(f"Platform title: {line.strip()}")
            
except Exception as e:
    print(f"Error processing gene annotation: {e}")


### Step 7: Gene Identifier Mapping

In [ ]:
# 1. Observe gene identifiers and gene annotation
# The gene identifiers in the expression data are Illumina probe IDs (e.g., "ILMN_1343291")
# These correspond to the "ID" column in the gene annotation dataframe
# The gene symbols are stored in the "Symbol" column of the gene annotation dataframe

# 2. Get gene mapping dataframe by extracting ID and Symbol columns
# Create a mapping dataframe
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')

# 3. Convert probe-level measurements to gene expression data
# Apply the gene mapping to convert probe IDs to gene symbols
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Print information about the gene data
print(f"Original gene expression data shape: {gene_data.shape}")
if not gene_data.empty:
    print(f"First 10 gene symbols: {list(gene_data.index[:10])}")
    sample_count = gene_data.shape[1]
    print(f"Number of samples: {sample_count}")
else:
    print("No genes mapped. Check if gene_annotation contains valid mapping information.")


### Step 8: Data Normalization and Linking

In [ ]:
# 1. Reload the gene expression data
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_data = get_genetic_data(matrix_file)

# Get gene mapping dataframe and apply it
gene_annotation = get_gene_annotation(soft_file)
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Symbol')
gene_data = apply_gene_mapping(gene_data, mapping_df)

# Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Check if trait data is available based on previous steps
# From Step 3, we determined that trait data is not available
is_trait_available = False

# Since there's no clinical data available, we'll create a minimal clinical dataframe with just the sample IDs
gene_samples = normalized_gene_data.columns
sample_df = pd.DataFrame(index=['dummy_trait'], columns=gene_samples)
sample_df.loc['dummy_trait'] = None  # Fill with NaN values
clinical_data = sample_df

print(f"Clinical data shape: {clinical_data.shape}")
print("No actual clinical trait data available for this cohort.")

# 3. Link clinical and genetic data (though clinical data is just a placeholder)
linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first few columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# Since there's no trait data, we cannot perform trait-based missing value handling
# or judge if the trait is biased. We'll just create a properly formatted dataframe.
cleaned_data = linked_data

# Remove the dummy trait column if it exists
if 'dummy_trait' in cleaned_data.columns:
    cleaned_data = cleaned_data.drop(columns=['dummy_trait'])

# 5. Validate and save cohort info
note = "This dataset contains gene expression data but does not contain the specific trait data for glucocorticoid sensitivity as required."
is_gene_available = len(normalized_gene_data) > 0

# Since there's no trait data, the dataset is not usable for associational studies
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=None,  # No trait to evaluate bias
    df=cleaned_data,
    note=note
)

# The dataset is not usable for our analyses since it lacks trait data
print("Data was determined to be unusable due to missing trait information and was not saved.")


### Step 9: Data Normalization and Linking

In [ ]:
# 1. Normalize gene symbols in the obtained gene expression data
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")
print(f"Sample gene symbols after normalization: {list(normalized_gene_data.index[:10])}")

# Save the normalized gene data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Gene data saved to {out_gene_data_file}")

# 2. Load the previously saved clinical data
clinical_data = pd.read_csv(out_clinical_data_file, index_col=0)
print(f"Clinical data shape: {clinical_data.shape}")
print("Clinical data preview:")
print(clinical_data.head())

# 3. Link clinical and genetic data using proper sample IDs
# First, transpose gene expression data to have samples as rows
gene_data_t = normalized_gene_data.T

# The clinical data should have samples as columns - verify sample IDs match
gene_samples = set(gene_data_t.index)
clinical_samples = set(clinical_data.columns)
common_samples = gene_samples.intersection(clinical_samples)

print(f"Gene samples: {len(gene_samples)}")
print(f"Clinical samples: {len(clinical_samples)}")
print(f"Common samples: {len(common_samples)}")

# Use the geo_link_clinical_genetic_data function to properly link the data
linked_data = geo_link_clinical_genetic_data(clinical_data, normalized_gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, first 5 columns):")
if linked_data.shape[1] >= 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data.head())

# 4. Handle missing values
print("\nMissing values before handling:")
print(f"  Trait ({trait}) missing: {linked_data[trait].isna().sum()} out of {len(linked_data)}")
gene_cols = [col for col in linked_data.columns if col not in [trait, 'Age', 'Gender']]
if gene_cols:
    missing_genes_pct = linked_data[gene_cols].isna().mean()
    genes_with_high_missing = sum(missing_genes_pct > 0.2)
    print(f"  Genes with >20% missing: {genes_with_high_missing}")
    
    if len(linked_data) > 0:  # Ensure we have samples before checking
        missing_per_sample = linked_data[gene_cols].isna().mean(axis=1)
        samples_with_high_missing = sum(missing_per_sample > 0.05)
        print(f"  Samples with >5% missing genes: {samples_with_high_missing}")

# Handle missing values
cleaned_data = handle_missing_values(linked_data, trait)
print(f"Data shape after handling missing values: {cleaned_data.shape}")

# 5. Evaluate bias in trait and demographic features
trait_biased, cleaned_data = judge_and_remove_biased_features(cleaned_data, trait)

# 6. Final validation and save
note = "Dataset contains gene expression data from glucocorticoid sensitivity studies. "
if 'Age' in cleaned_data.columns:
    note += "Age data is available. "
if 'Gender' in cleaned_data.columns:
    note += "Gender data is available. "

is_gene_available = len(normalized_gene_data) > 0
is_usable = validate_and_save_cohort_info(
    is_final=True, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available, 
    is_trait_available=True, 
    is_biased=trait_biased, 
    df=cleaned_data,
    note=note
)

# 7. Save if usable
if is_usable and len(cleaned_data) > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    cleaned_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Data was determined to be unusable or empty and was not saved")